# Collaboration Filtering Model for Movie Recommendation
## Oleh : Febi Andika Dani Fajar Suryawan

## Studi Kasus
Diberikan data berisi 10 film yang telah diberi rating oleh 24 user. Akan dibuat daftar rekomendasi film kepada seorang user yang belum menonton suatu film berdasarkan pada kemiripan penilaian rating film dari user yang lain. 

## Preparasi Data

In [1]:
# Memanggil library yang dibutuhkan
#from recommendation_data import dataset
from math import sqrt
import pandas as pd

In [15]:
# Memanggil dataset
data=pd.read_csv('Film_Rating.csv',delimiter=';')
data=data.set_index('Nama Anda')
data.head()

,Ada Apa dengan Cinta 2,Gundala,Dilan 1991,Bumi Manusia,Dua Garis Biru,Avengers: End Game,The Lion King,Aladdin,Spiderman: Far From Home,Captain Marvel
Nama Anda,,,,,,,,,,
Hania,3,5,4,4,4,0,0,0,0,0
Topik Zulkarnain,0,0,0,0,0,5,5,0,4,2
AhokTemanFirli,0,0,0,0,0,3,0,0,0,4
franadek,4,4,4,5,3,5,4,5,4,4
OM INDRA,3,0,2,0,5,5,0,1,5,5


In [13]:
# Membuat data menjadi dictionary
dataset=data.to_dict(orient='index')
for name in dataset:
    dicto={}
    for movie in dataset[name]:
        if dataset[name][movie]!=0:
            dicto[movie]=dataset[name][movie]
    dataset[name]=dicto

## Collaboration Filtering

### Similarity Score
Pada penghitungan similarity score akan digunakan jarak euclidean dari rating yang diberikan dari dua orang yang memberikan rating terhadap suatu film. Nilai similarity score sendiri adalah 1/(1+euclidean).

In [29]:
# Membuat fungsi untuk menghitung nilai euclidean distance dari orang pertama dan kedua
def similarity_score(person1,person2):
    
    # Mengambil item yang telah diberi rating oleh orang pertama dan orang kedua
    both_viewed = {}

    for item in dataset[person1]:
        if item in dataset[person2]:
            both_viewed[item] = 1
            
        if len(both_viewed) == 0:
            return 0

        # Menghitung Euclidean distance
        sum_of_eclidean_distance = []

        for item in dataset[person1]:
            if item in dataset[person2]:
                sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item], 2))
        sum_of_eclidean_distance = sum(sum_of_eclidean_distance)
        
        return 1/(1+sqrt(sum_of_eclidean_distance))


### Person Correlation
Pada penghitungan person correlation akan digunakan korelasi diantara dua orang yang memberikan rating terhadap suatu film. Semakin besar nilai person correlation berarti kedua orang memiliki preferensi/ kesukaan yang sama terhadap suatu film, begitupun sebaliknya.

In [9]:
# Membuat fungsi untuk menghitung nilai person correlation dari orang pertama dan kedua
def person_correlation(person1, person2):

   # Mengambil item yang keduanya telah diberi rating
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    if number_of_ratings == 0:
        return 0

    # Menjumlahkan semua preferensi dari setiap user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Jumlah kuadrat dari preferensi dari setiap user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Menjumlahkan nilai produk dari preferensi keduanya untuk setiap item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Menghitung pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

## Model Rekomendasi Film
Pada model rekomendasi film berikut digunakan person correlation untuk memberikan score/nilai rekomendasi terhadap suatu film.

In [10]:
# Fungsi untuk menentukan banyaknya users (orang yang mirip) dengan orang tersebut
def most_similar_users(person, number_of_users):

    scores = [(person_correlation(person, other_person), other_person) for other_person in dataset if other_person != person]
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]

In [11]:
# Membuat fungsi yang memberikan rekomendasi film kepada seseorang
def user_recommendations(person):

    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        if other == person:
            continue
        sim = person_correlation(person,other)
        if sim <=0: 
            continue
        for item in dataset[other]:

            # Hanya menilai film yang belum ditonton
            if item not in dataset[person] or dataset[person][item] == 0:
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                simSums.setdefault(item,0)
                simSums[item]+= sim

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    recommendations_list = [recommend_item for score,recommend_item in rankings]
    return recommendations_list,rankings

Sebagai contoh, akan dicari rekomendasi film yang belum ditonton untuk user bernama ANI. Rekomendasi ini diurutkan berdasarkan recommendation score yang tertinggi hingga kerendah. Diperoleh list rekomendasi film untuk ANI adalah sebagai berikut:

In [14]:
# Input berupa nama user
name = input("Masukkan Nama: ")

# Memanggil fungsi untuk merekomendasikan film kepada user
recommendation=user_recommendations(name)
out=pd.DataFrame(recommendation[1],columns=['Recommendation Score','Film Title'])
out=out[['Film Title','Recommendation Score']]

# Mengeluarkan rekomendasi film yang belum ditonton, yang disarankan berdasarkan recommendation score
print('Film yang direkomendasikan untuk {} adalah'.format(name))
print(out)

Masukkan Nama: ANI
Film yang direkomendasikan untuk ANI adalah
           Film Title  Recommendation Score
0  Avengers: End Game              5.000000
1             Gundala              4.265556
2       The Lion King              4.179129
3      Dua Garis Biru              3.620204


**Catatan :** Dalam membuat model collaborative filtering, alangkah baiknya dibuat dengan menggunakan data yang memiliki respon rating dari user yang banyak dan jumlah film yang banyak juga. Hal ini dapat membuat proses pembuatan rekomendasi film lebih baik. Pada model ini hanya digunakan data yang diperoleh dari hasil pembelajaran dikelas.